In [57]:
import requests
from bs4 import BeautifulSoup
import os


In [58]:
url = 'https://nanohub.org/citations' #url we want to pull the images from and save under plots_production  
doc = requests.get(url) # beautiful soup can't work with http requests directly so must get document behind the link and feed that to beautiful soup 
citationPage = BeautifulSoup(doc.text, 'html')
images = citationPage.find_all('img') # extract all the image tags

# lists to extract appropirate info
imageLinks = []
imageDesc = []
csvFiles = []
for link in citationPage.find_all('a'):
    potentialcsv = link.get('href')
    if potentialcsv is not None and potentialcsv.endswith('.csv'):
        csvFiles.append("http://nanohub.org" + potentialcsv)
print(csvFiles)
for files in csvFiles:
    download = requests.get(files)
    names = files.split('/')
    with open(names[-1], 'wb') as f:
        f.write(download.content)
        os.rename(names[-1], './pulledcsv/' + names[-1]) # saves all web scraped csv files to a pulled csv folder 
imageLinks = [image['src'] for image in images] # extract the links to all the images we want to save
imageDesc = [image['alt'] for image in images] #extract any description associated with the images so we can have an easier time naming 
imageLinks = ["http://nanohub.org" + image for image in imageLinks] #append nanohub.org to get the actual not just the partial link 
imgCount = 0 # used for naming which citation image is saved 
for toSave in imageLinks:
    with open('Citationimg' + str(imgCount) + '.svg', 'wb') as f:
        filename = 'Citationimg' + str(imgCount) + '.svg'
        citationimg = requests.get(toSave)
        f.write(citationimg.content) # .content gets the individual byte information and saves that 
        os.rename(filename, './plots_production/' + filename) # moves the files to the appropriate place 
    imgCount += 1


['http://nanohub.org/app/site/stats/citation_maps/citations_report.csv']
